A simple Travelling Salesman Problem to solve (optimize) with IBM Cplex for learning purposes

_"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"_

*(Wikipedia.org, 2023)* - https://en.wikipedia.org/wiki/Travelling_salesman_problem

![problem-studied](./documentation/problem.png)

# BEFORE CODING: INSTALL CPLEX AND SETUP YOUR PYTHON ENVIRONEMENT
0. Make sure you have Python **3.10* installed (brew install python@3.10 on MacOs) because: _Cplex currently do not support newer versions!_
1. Download Cplex from **IBM official website** and install it: https://www.ibm.com/products/ilog-cplex-optimization-studio/cplex-optimizer
2. Go to `CPLEX_HOME/cplex/python/3.10/x86-64_osx/` or `CPLEX_HOME/cplex/python/3.10/arm64_osx/` on MacOs
    * --> _For example (on MacOs):_ **CPLEX_HOME**=`/Applications/CPLEX_Studio2211`
3. Run:
    * `export CPLEX_STUDIO_DIR="/Applications/CPLEX_Studio2211"`
    *  `export DYLD_LIBRARY_PATH="$CPLEX_STUDIO_DIR/cplex/bin/arm64_osx:$DYLD_LIBRARY_PATH"`
    * `export PATH="$CPLEX_STUDIO_DIR/cplex/bin/arm64_osx:$PATH"`
4. (Option 1 old) Run: `python3.10 setup.py install --home PATH_TO_PYTHON_PACKAGES/cplex`
    * --> _For example (on MacOs):_ **PATH_TO_PYTHON_PACKAGES**=`/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages`
4. (Option 2 new) Run: `pip install --upgrade pip setuptools wheel` and then `pip3.10 install .`
5. Run: `pip3.10 install numpy cplex docplex`

In [3]:
# 0.A IMPORT ALL USED LIBRARIES
from random import randint
import numpy as np

In [4]:
# 0.B CONFIGURE THE PROBLEM
NBR_CITIES = 5
MIN_DISTANCE_BETWEEN_CITIES = 1
MAX_DISTANCE_BETWEEN_CITIES = 15

# 0.C CONFIGURE CPLEX
COMPUTING_TIME_LIMIT = 300 #seconds
MEMORY_LIMIT = 1024 #in MB
NODE_LIMIT = 1000 #size of the branch&bound/cut decision tree
MIP_TOLERANCE = 0 #%
MAX_NBR_THREADS = 4

In [5]:
# I. BUILD A RANDOM TSP INSTANCE 
def build_random_instance(nbr_cities=6, min_distance=1, max_distance=15):
    start = randint(0, nbr_cities-1) # 1. random generate the starting city
    distances = np.random.uniform(min_distance, max_distance, (nbr_cities, nbr_cities)).astype(int) # 2. random generate a matrix of (integer) paths between cities
    distances = np.triu(distances, k=1)
    distances += distances.T # 3. the distance between city a -> city b should be equal to the between city b -> city a
    np.fill_diagonal(distances, 0) # 4. replace all distance for a city to itself (the diagonal) to 0 
    return distances, start

distances, start = build_random_instance(NBR_CITIES, MIN_DISTANCE_BETWEEN_CITIES, MAX_DISTANCE_BETWEEN_CITIES)
print(distances)
print(start)

[[ 0 10  3  1  3]
 [10  0  4 11  2]
 [ 3  4  0  7  4]
 [ 1 11  7  0 14]
 [ 3  2  4 14  0]]
1


In [7]:
from docplex.mp.model import Model

# II. BUILD A DOCPLEX MODEL WITH DECISION VARIABLES, FIRST TWO CONSTRAINTS, AND OBJECTIVE
def build_model(distances, nbr_cities=6):
    m = Model(name="TSP_DFJ")

    # 1) Create an array of binary variables path_i_j for all i != j
    arcs = [(i, j) for i in range(nbr_cities) for j in range(nbr_cities) if j != i]
    x = m.binary_var_dict(arcs, name="path")  # vars will be named like path_0_1, ...

    # 2) Minimize total travel distance
    m.minimize(m.sum(int(distances[i][j]) * x[i, j] for (i, j) in arcs))

    # 3) Degree constraints: exactly one path FROM and TO each city
    for i in range(nbr_cities):
        # from i: sum_j x[i,j] = 1
        m.add_constraint(m.sum(x[i, j] for j in range(nbr_cities) if j != i) == 1, ctname=f"out_{i}")
        # to i: sum_j x[j,i] = 1
        m.add_constraint(m.sum(x[j, i] for j in range(nbr_cities) if j != i) == 1, ctname=f"in_{i}")

    # 4) Apply runtime configuration (same values; note: mipgap expects a fraction)
    m.context.cplex_parameters.timelimit = COMPUTING_TIME_LIMIT              
    m.context.cplex_parameters.workmem = MEMORY_LIMIT                       
    m.context.cplex_parameters.mip.limits.nodes = NODE_LIMIT
    m.context.cplex_parameters.mip.tolerances.mipgap = MIP_TOLERANCE
    m.context.cplex_parameters.threads = MAX_NBR_THREADS
    return m

model = build_model(distances, NBR_CITIES)

![calcul-of-subsets](./documentation/calcul.png)

In [9]:
from itertools import combinations

# III. DFJ CONSTRAINTS TO AVOID SUB-TOURS (O(2^n) CONSTRAINTS)
def generate_subsets(nbr_cities:int=6):
    subsets = []
    for size in range(2,nbr_cities):
        subsets.extend(combinations(range(nbr_cities), size))
    return subsets

def add_last_constraint(m, nbr_cities: int = 6):
    for S in generate_subsets(nbr_cities):
        # sum_{i in S} sum_{j in S, j != i} x[i,j] <= |S| - 1
        paths = [f"path_{i}_{j}" for i in S for j in S if j != i]
        print(paths)
        print(len(S) - 1)
        expr = m.sum(m.get_var_by_name(var_name) for var_name in paths)
        m.add_constraint(expr <= len(S) - 1, ctname=f"dfj_{'_'.join(map(str, S))}")
    return m

model = add_last_constraint(model, NBR_CITIES)

['path_0_1', 'path_1_0']
1
['path_0_2', 'path_2_0']
1
['path_0_3', 'path_3_0']
1
['path_0_4', 'path_4_0']
1
['path_1_2', 'path_2_1']
1
['path_1_3', 'path_3_1']
1
['path_1_4', 'path_4_1']
1
['path_2_3', 'path_3_2']
1
['path_2_4', 'path_4_2']
1
['path_3_4', 'path_4_3']
1
['path_0_1', 'path_0_2', 'path_1_0', 'path_1_2', 'path_2_0', 'path_2_1']
2
['path_0_1', 'path_0_3', 'path_1_0', 'path_1_3', 'path_3_0', 'path_3_1']
2
['path_0_1', 'path_0_4', 'path_1_0', 'path_1_4', 'path_4_0', 'path_4_1']
2
['path_0_2', 'path_0_3', 'path_2_0', 'path_2_3', 'path_3_0', 'path_3_2']
2
['path_0_2', 'path_0_4', 'path_2_0', 'path_2_4', 'path_4_0', 'path_4_2']
2
['path_0_3', 'path_0_4', 'path_3_0', 'path_3_4', 'path_4_0', 'path_4_3']
2
['path_1_2', 'path_1_3', 'path_2_1', 'path_2_3', 'path_3_1', 'path_3_2']
2
['path_1_2', 'path_1_4', 'path_2_1', 'path_2_4', 'path_4_1', 'path_4_2']
2
['path_1_3', 'path_1_4', 'path_3_1', 'path_3_4', 'path_4_1', 'path_4_3']
2
['path_2_3', 'path_2_4', 'path_3_2', 'path_3_4', 'path_

In [11]:
# IV. PRINT THE FINAL MODEL (DOCPLEX VERSION)
print("\nConstraints:")
for ct in model.iter_constraints():
    expr = ct.lhs
    sense = ct.sense  # 'L', 'G', or 'E'
    rhs = ct.rhs
    sense_symbol = {"L": "<=", "G": ">=", "E": "="}.get(sense, "?")

    # Build string version of expression (sum of variable names)
    var_terms = []
    for term, coef in expr.iter_terms():
        name = term.name if hasattr(term, "name") else str(term)
        if coef == 1:
            var_terms.append(f"{name}")
        else:
            var_terms.append(f"{coef}*{name}")
    constraint_expr = " + ".join(var_terms)
    print(f"{constraint_expr} {sense_symbol} {rhs}")

print("\nObjective:")
obj_expr = model.objective_expr
obj_terms = []
for term, coef in obj_expr.iter_terms():
    name = term.name if hasattr(term, "name") else str(term)
    obj_terms.append(f"{coef}*{name}")
print("Minimize " + " + ".join(obj_terms))



Constraints:
path_0_1 + path_0_2 + path_0_3 + path_0_4 ? 1
path_1_0 + path_2_0 + path_3_0 + path_4_0 ? 1
path_1_0 + path_1_2 + path_1_3 + path_1_4 ? 1
path_0_1 + path_2_1 + path_3_1 + path_4_1 ? 1
path_2_0 + path_2_1 + path_2_3 + path_2_4 ? 1
path_0_2 + path_1_2 + path_3_2 + path_4_2 ? 1
path_3_0 + path_3_1 + path_3_2 + path_3_4 ? 1
path_0_3 + path_1_3 + path_2_3 + path_4_3 ? 1
path_4_0 + path_4_1 + path_4_2 + path_4_3 ? 1
path_0_4 + path_1_4 + path_2_4 + path_3_4 ? 1
path_0_1 + path_1_0 ? 1
path_0_2 + path_2_0 ? 1
path_0_3 + path_3_0 ? 1
path_0_4 + path_4_0 ? 1
path_1_2 + path_2_1 ? 1
path_1_3 + path_3_1 ? 1
path_1_4 + path_4_1 ? 1
path_2_3 + path_3_2 ? 1
path_2_4 + path_4_2 ? 1
path_3_4 + path_4_3 ? 1
path_0_1 + path_0_2 + path_1_0 + path_1_2 + path_2_0 + path_2_1 ? 2
path_0_1 + path_0_3 + path_1_0 + path_1_3 + path_3_0 + path_3_1 ? 2
path_0_1 + path_0_4 + path_1_0 + path_1_4 + path_4_0 + path_4_1 ? 2
path_0_2 + path_0_3 + path_2_0 + path_2_3 + path_3_0 + path_3_2 ? 2
path_0_2 + pat

In [14]:
# V. RUN THE MODEL AND DISPLAY THE FINAL SOLUTION (DOCPLEX VERSION)
solution = model.solve()

if solution is None:
    print("No feasible solution found.")
else:
    try:
        print("OBJECTIVE FUNCTION VALUE: ", solution.get_objective_value())
    except AttributeError:
        print("OBJECTIVE FUNCTION VALUE: ", model.objective_value)

    # 2) Dumb display of final value of each decision variable
    def dumb_display(m):
        for v in m.iter_variables():
            val = v.solution_value
            if val is not None and val >= 1:
                print(f"{v.name}: {val}")

    dumb_display(model)

    # 3) Smart display the complete selected path
    def search_next_city(var_names, var_values, current_city=0, nbr_cities=6):
        paths_from_current_city = {f"path_{current_city}_{i}" for i in range(nbr_cities) if i != current_city}
        for name, value in zip(var_names, var_values):
            if name in paths_from_current_city and value is not None and value >= 1:
                return int(name.split("_")[2])
        return -1

    def display_path(m, distances, start:int=0, nbr_cities:int=6):
        var_names  = [v.name for v in m.iter_variables()]
        var_values = [v.solution_value for v in m.iter_variables()]
        first_itr = True
        current_city = start
        next_city = -1
        print("COMPLETE PATH:", end=' ')
        while first_itr or (next_city != -1 and current_city != start):
            next_city = search_next_city(var_names, var_values, current_city, nbr_cities)
            if next_city != -1:
                print(f"City_{current_city} -> ({int(distances[current_city][next_city])}) ->", end=' ')
                current_city = next_city
            else:
                print(f"City_{current_city}", end=' ')
            first_itr = False
        print(f"City_{start}")

    display_path(model, distances, start, NBR_CITIES)


OBJECTIVE FUNCTION VALUE:  17.0
path_0_4: 1.0
path_1_2: 1.0
path_2_3: 1.0
path_3_0: 1.0
path_4_1: 1.0
COMPLETE PATH: City_1 -> (4) -> City_2 -> (7) -> City_3 -> (1) -> City_0 -> (3) -> City_4 -> (2) -> City_1
